# Notebook d'entrainement d'un modèle personnalisé

### Import des dépendances

In [6]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp_fr = spacy.load("fr_core_news_md")

### Test de reconnaissance de NER sur un texte simple

In [7]:
texte_simple = '''Je voudrais aller de Toulouse à Bordeaux.
Comment me rendre à Port-Boulet depuis la gare de Tours ?
Je veux aller voir mon ami Albert à Tours en partant de Bordeaux.'''
doc = nlp_fr(texte_simple)

spacy.displacy.render(doc, style="ent", jupyter=True)

Le modèle de base reconnait les NER "LOC" (localisation). Le but maintenant est d'entrainer un modèle qui puisse aller plus loin en reconnaissant si le NER est un départ ou une destination.

### Chargement du jeu de données pour l'entrainement

Pour l'entrainement, nous utilisons un dataset composé de plusieurs phrases.
La ville de départ (DEPARTURE) et la ville de destination (DESTINATION) sont identifiées pour chacune des phrases.

In [8]:
import json
f = open('./datasets/training_data.json')
TRAIN_DATA = json.load(f)
TRAIN_DATA

{'classes': ['DEPARTURE', 'DESTINATION', 'ESCALE'],
 'annotations': [['Je voudrais aller de Toulouse à Bordeaux.\r',
   {'entities': [[21, 29, 'DEPARTURE'], [32, 40, 'DESTINATION']]}],
  ['Comment me rendre à Port-Boulet depuis la gare de Tours ?\r',
   {'entities': [[20, 31, 'DESTINATION'], [50, 55, 'DEPARTURE']]}],
  ['Je veux aller voir mon ami Albert à Tours en partant de Bordeaux.\r',
   {'entities': [[36, 41, 'DESTINATION'], [56, 64, 'DEPARTURE']]}],
  ['Y a-t-il des trains de Nantes à Montaigu ?\r',
   {'entities': [[23, 29, 'DEPARTURE'], [32, 40, 'DESTINATION']]}],
  ['Une phrase sans origine ni destination.\r', {'entities': []}],
  ["Si pas de numéro de séquence, on considère que c'est zéro\r",
   {'entities': []}],
  ['Is there any train going from Paris to Marseille ?\r', {'entities': []}],
  [' Je souhaite me rendre à Paris en partant de Toulouse\r',
   {'entities': [[25, 30, 'DESTINATION'], [45, 53, 'DEPARTURE']]}],
  [' Je quitte Lyon pour aller à Marseille\r',
   {'entit

In [9]:
# !python -m spacy download fr_core_news_md # Décommenter pour installer le modèle

### Conversion du dataset au format attendu par spaCy

In [10]:
from random import shuffle


db = DocBin() # Le DocBin est un conteneur pour les documents dans spaCy

# Mélange des données pour éviter un apprentissage biaisé
shuffle(TRAIN_DATA['annotations'])

# Conversion des données au format spaCy
for text, annot in tqdm(TRAIN_DATA['annotations']): 
    doc = nlp_fr.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents 
    db.add(doc)

db.to_disk("./training_data.spacy")

100%|██████████| 148/148 [00:00<00:00, 3363.57it/s]


### Entrainement du modèle

In [11]:
# Cette commande permet de créer un fichier de configuration pour l'entraînement
! python -m spacy init config config.cfg --lang fr --pipeline ner --optimize efficiency --force

ℹ Generated config template specific for your use case
- Language: fr
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [12]:
# Lancement de l'entrainement
! python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     57.00    0.00    0.00    0.00    0.00
  9     200         96.12   1601.36   96.34   95.95   96.73    0.96
 21     400         82.23    192.39   99.18   99.18   99.18    0.99
 35     600         55.33     90.84   99.59   99.59   99.59    1.00
 53     800         70.71     56.23   99.59   99.59   99.59    1.00
 75    1000         43.39     47.48   99.59   99.59   99.59    1.00
102    1200        120.32     74.60   99.39   99.19   99.59    0.99
134    1400        108.38     76.01   99.59   99.59   99.59    1.00
173    1600         64.48     76.43 

### Chargement et test du modèle

In [21]:
# Chargement du modèle customisé
nlp_itineraire = spacy.load("./model-best/")

# Nous définissons ici les couleurs pour les entités (utilisées lors du rendu avec displacy)
colors = {"DEPARTURE": "#ffe899", "DESTINATION": "#b1ff5e", "ESCALE": "#82b8ff"}
options = {"ents": ["DEPARTURE", "DESTINATION", "ESCALE"], "colors": colors}

Identification des NER avec le modèle entrainé

In [22]:
doc2 = nlp_itineraire(texte_simple)
spacy.displacy.render(doc2, style="ent", options=options, jupyter=True)

Pour mémoire : identification des NER avec le modèle de base.

In [23]:
doc = nlp_fr(texte_simple)
spacy.displacy.render(doc, style="ent", jupyter=True)

Test du modèle sur un texte plus compliqué

In [24]:
texte_complique = '''Je souhaite me rendre à Lille en partant d'Aubervilliers pour assister à une conférence.
Je compte me rendre à Bordeaux depuis Marseille pour rendre visite à ma soeur Paris.
Je dois regarder les trains Toulouse - Brest pour aller voir mon ami Albert.
Je dois planifier un voyage Nice Toulouse pour les prochaines vacances.
Une réunion de travail m'oblige à faire Paris - Clermont-Ferrand la semaine prochaine.'''
doc3 = nlp_itineraire(texte_complique) # input sample text

spacy.displacy.render(doc3, style="ent", options=options, jupyter=True)

Test du modèle sur un texte comprenant des escales

In [25]:
texte_escale = '''Je souhaite me rendre à Lille en partant d'Aubervilliers pour assister à une conférence avec une escale à Nice.
Je compte me rendre à Bordeaux depuis Marseille en m'arrêtant à Toulouse pour rendre visite à ma soeur Paris.
Je dois regarder les trains Toulouse - Brest pour aller voir mon ami Albert en faisant une escale à Tours.
Je dois planifier un voyage Nice Toulouse en passant par Aubervilliers pour les prochaines vacances.
Une réunion de travail m'oblige à faire Paris - Clermont-Ferrand avec un arrêt par Lyon la semaine prochaine.'''
doc4 = nlp_itineraire(texte_escale) # input sample text

spacy.displacy.render(doc4, style="ent", options=options, jupyter=True)